In [9]:
import requests
import json
import pandas as pd
from IPython.display import display, Markdown

API_BASE_URL = "http://localhost:8000"

In [10]:
def call_api(endpoint, method="get", data=None):
    url = f"{API_BASE_URL}{endpoint}"

    if method.lower() == "get":
        response = requests.get(url, params=data)
    elif method.lower() == "post":
        response = requests.post(url, json=data)
    else:
        raise ValueError(f"Unsupported HTTP method: {method}")

    if response.status_code >= 400:
        print(f"Error {response.status_code}: {response.text}")
        return None

    return response.json()

In [11]:
# Test 1: API Status
status = call_api("/")
print("API Status:")
print(json.dumps(status, indent=2))

API Status:
{
  "status": "ok",
  "message": "Game recommender API is running",
  "model_info": {
    "n_users": 964,
    "n_items": 11879,
    "n_factors": 5
  }
}


In [12]:
# Test 2: Add ratings for new user
ratings_data = {
    "ratings": [
        {"BGGId": 174430, "Rating": 8.5},  # Gloomhaven
        {"BGGId": 167791, "Rating": 7.2},  # Terraforming Mars
        {"BGGId": 169786, "Rating": 9.0},  # Scythe
        {"BGGId": 233078, "Rating": 6.8},  # Wingspan
        {"BGGId": 162886, "Rating": 8.3}   # Spirit Island
    ],
    "UserId": None  # New user
}

ratings_response = call_api("/ratings", method="post", data=ratings_data)
print("Add Ratings Response:")
print(json.dumps(ratings_response, indent=2))

user_id = ratings_response.get("user_id")

Add Ratings Response:
{
  "success": true,
  "user_id": 344224,
  "is_new_user": true,
  "num_ratings_processed": 5
}


In [13]:
# Test 3: Get personalized Recommendations
recommendation_request = {
    "user_id": user_id,
    "num_recommendations": 20,
    "include_details": True
}

recommendations = call_api("/recommendations", method="post", data=recommendation_request)
print(f"Recommendations for user {user_id}:")

if recommendations:
    recommendations_df = pd.DataFrame(recommendations)
    display(recommendations_df)

Recommendations for user 344224:


,BGGId,PredictedRating,Name,YearPublished,Description,ImageURL
0,266507,8.975883,Clank!: Legacy – Acquisitions Incorporated,2019,clank legacy acquisition incorporate extend de...,None
1,246900,8.921331,Eclipse: Second Dawn for the Galaxy,2020,game eclipse place control vast interstellar c...,None
2,169427,8.910458,Middara: Unintentional Malum – Act 1,2019,middara player cooperative chooseyourownadve...,None
3,270633,8.838820,Aeon's End: The New Age,2019,aeonrsquo end new age introduce expedition sys...,None
4,224517,8.787398,Brass: Birmingham,2018,brass birmingham economic strategy game sequel...,None
5,284378,8.786235,Kanban EV,2020,electric vehicle evs common future automobil...,None
6,281655,8.756196,High Frontier 4 All,2020,th ed phil eklund signature game high frontier...,None
7,255984,8.721951,Sleeping Gods,2021,quotare star unfamiliar herequot ask sky grow ...,None
8,161936,8.702958,Pandemic Legacy: Season 1,2015,pandemic legacy cooperative campaign game over...,None
9,291457,8.694219,Gloomhaven: Jaws of the Lion,2020,gloomhaven jaw lion standalone game take place...,None


In [15]:
# Test 4: Get Popular Recommendations
recommendation_request = {
    "num_recommendations": 20,
    "include_details": True
}

popular_response = call_api("/popular", data=recommendation_request)
print(f"Popular recommendations for user {user_id}:")
print("Popular Games:")

if popular_response:
    popular_df = pd.DataFrame(popular_response)
    display(popular_df)

Popular recommendations for user 344224:
Popular Games:


,BGGId,PredictedRating,Name,YearPublished,Description,ImageURL
0,167791,46.589354,Terraforming Mars,2016,s mankind begin terraform planet mar giant cor...,None
1,68448,45.071618,7 Wonders,2010,leader great city ancient world gather resou...,None
2,169786,44.913206,Scythe,2016,time unrest s europa ashe great war darken sno...,None
3,30549,44.883919,Pandemic,2008,pandemic virulent disease break simultaneously...,None
4,822,44.289854,Carcassonne,2000,carcassonne tileplacement game player draw pla...,None
5,173346,43.926876,7 Wonders Duel,2015,way wonder duel resemble parent game wonde...,None
6,161936,43.883380,Pandemic Legacy: Season 1,2015,pandemic legacy cooperative campaign game over...,None
7,174430,43.811290,Gloomhaven,2017,gloomhaven game euroinspired tactical combat...,None
8,84876,43.657031,The Castles of Burgundy,2011,game set burgundy region high medieval france ...,None
9,3076,43.338779,Puerto Rico,2002,puerto rico player assume role colonial govern...,None
